In [5]:
from typing import Dict
from sklearn.model_selection import train_test_split

from dataclasses import dataclass, field
from typing import cast, Optional

import torch
from datasets import load_dataset
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)
from transformers.trainer_pt_utils import LabelSmoother
from transformers.integrations import deepspeed

from peft import PeftConfig, PeftModel

import os
IGNORE_TOKEN_ID = LabelSmoother.ignore_index
import sys
sys.path.append('/home/jupyter/TQL/')
# from utils.token import hub_token
import pandas as pd
import numpy as np


def format_example(example):
    instruction = (
        "Generate an accurate SQL query to answer the following question using only the given tables: "
    )
    q_header = "### Question"
    a_header = "### Answer"

    q = example

    return f"{instruction}\n\n{q_header}\n{q}\n\n{a_header}\n"

def get_gpu_memory(max_gpus=None):
    """Get available memory for each GPU."""
    import torch

    gpu_memory = []
    num_gpus = (
        torch.cuda.device_count()
        if max_gpus is None
        else min(max_gpus, torch.cuda.device_count())
    )

    for gpu_id in range(num_gpus):
        with torch.cuda.device(gpu_id):
            device = torch.cuda.current_device()
            gpu_properties = torch.cuda.get_device_properties(device)
            total_memory = gpu_properties.total_memory / (1024**3)
            allocated_memory = torch.cuda.memory_allocated() / (1024**3)
            available_memory = total_memory - allocated_memory
            gpu_memory.append(available_memory)
    return gpu_memory

def load_model(model_path, num_gpus, max_gpu_memory=None):
    
    kwargs = {"torch_dtype": torch.float16}
    if num_gpus != 1:
        kwargs["device_map"] = "auto"
        if max_gpu_memory is None:
            kwargs[
                "device_map"
            ] = "sequential"  # This is important for not the same VRAM sizes
            available_gpu_memory = get_gpu_memory(num_gpus)
            kwargs["max_memory"] = {
                i: str(int(available_gpu_memory[i] * 0.85)) + "GiB"
                for i in range(num_gpus)
            }
        else:
            kwargs["max_memory"] = {i: max_gpu_memory for i in range(num_gpus)}
        
        config = PeftConfig.from_pretrained(model_path)
        base_model_path = config.base_model_name_or_path
        tokenizer = AutoTokenizer.from_pretrained(
            base_model_path, use_fast=False
        )
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            low_cpu_mem_usage=True,
            **kwargs,
        )
        model = PeftModel.from_pretrained(base_model, model_path)
        
        return model, tokenizer, base_model

model, tokenizer, base_model = load_model('../../../New_Model', torch.cuda.device_count(), max_gpu_memory=None)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
queryData = pd.read_csv('sql_queries.csv')
question = queryData['context'][100]
inputs = queryData['input'][100]
sql = queryData['SQL'][100]

prompt = format_example(inputs + ' ' + question)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])
print('-'*100)
print("Actual Query: ", sql)

The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM',

Generate an accurate SQL query to answer the following question using only the given tables: 

### Question
What are the ids of the students who either registered or attended a course? CREATE TABLE addresses (address_id number, line_1 text, line_2 text, city text, zip_postcode text, state_province_county text, country text) which has address_id as primary key and CREATE TABLE people (person_id number, first_name text, middle_name text, last_name text, cell_mobile_number text, email_address text, login_name text, password text) which has person_id as primary key and CREATE TABLE students (student_id number, student_details text) which has student_id as primary key and CREATE TABLE courses (course_id text, course_name text, course_description text, other_details text) which has course_id as primary key and CREATE TABLE people addresses (person_address_id number, person_id number, address_id number, date_from time, date_to time) which has person_address_id as primary key and CREATE TABLE 

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 387, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
